In [1]:
import pandas as pd

In [4]:
from langchain_core.documents import Document

sources = pd.read_excel("../results/FullStudyAnalysis.xlsx")
finalResult = []
for idx, source in sources.iterrows():
    row = {"doi":source['doi']}
    if  isinstance(source['uses_data_limits_sources'],str):
        documentObject = eval(source['uses_data_limits_sources'])
        i = 0
        for document in documentObject:
            i = i + 1
            row['Source'+str(i)] = document.metadata['source']
        finalResult.append(row)
pdResults = pd.DataFrame(finalResult)

In [2]:
from SourcesExtractor import extractor
dataPapersList = pd.read_excel("../results/FullStudyAnalysis.xlsx", sheet_name="Raw Data")
sources = extractor(dataPapersList)

/home/joangi/sites/DataPaperAnalysis/DataPaperAnalysis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/joangi/sites/DataPaperAnalysis/DataPaperAnalysis/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/joangi/sites/DataPaperAnalysis/DataPaperAnalysis/.venv/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use 

load INSTRUCTOR_Transformer
max_seq_length  512


/home/joangi/sites/DataPaperAnalysis/DataPaperAnalysis/.venv/lib/python3.12/site-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

Processing the: 0
Creating embeddings
PDF not parsed
0 processed!
Processing the: 1
Creating embeddings
PDF not parsed
1 processed!
Processing the: 2
Creating embeddings
PDF not parsed
2 processed!
Processing the: 3
Creating embeddings
PDF not parsed
3 processed!
Processing the: 4
Creating embeddings
PDF not parsed
4 processed!
Processing the: 5
Processing the: 6
Processing the: 7
Processing the: 8
Processing the: 9
Processing the: 10
Processing the: 11
Processing the: 12
Processing the: 13
Processing the: 14
Processing the: 15
Processing the: 16
Processing the: 17
Processing the: 18
Processing the: 19
Processing the: 20
Processing the: 21
Processing the: 22
Processing the: 23
Processing the: 24
Processing the: 25
Processing the: 26
Processing the: 27
Processing the: 28
Processing the: 29
Processing the: 30
Processing the: 31
Processing the: 32
Processing the: 33
Processing the: 34
Processing the: 35
Processing the: 36
Processing the: 37
Processing the: 38
Processing the: 39
Processing

In [15]:
dataPapersList["doi"].head()

0        https://doi.org/10.1038/sdata.2016.35
1        https://doi.org/10.1038/sdata.2015.66
2       https://doi.org/10.1038/sdata.2018.214
3       https://doi.org/10.1038/sdata.2017.122
4    https://doi.org/10.1038/s41597-020-0453-3
Name: doi, dtype: object